In [1]:
import numpy as np
from pyscf import gto, scf, fci
from numba import prange, jit, njit
#from numba.openmp import openmp_context as openmp

In [2]:
def expand_matrix(P):
    Paa = P
    Pbb = P
    Pab = np.zeros(P.shape)
    Pba = np.zeros(P.shape)
    PE = np.concatenate((np.concatenate((Paa, Pab), axis=1), np.concatenate((Pba, Pbb), axis=1)), axis=0) 
    return PE

In [3]:
# for jit and prange
@jit(nopython=True, parallel=True)
def ONERDMFT_hartree_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy

In [4]:
# for spinorbitals
@jit(nopython=True, parallel=True)
def ONERDMFT_Umrigar_hartree_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M):
        for b in [b for b in range(0,M) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy
#                if not(a == b):


In [5]:
@jit(nopython=True, parallel=True)
def ONERDMFT_Umrigar_exchange_correlation_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M//2):
        for b in [b for b in range(0,M//2) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2,M):
        for b in [b for b in range(M//2,M) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    

    return -1/2.*energy

In [6]:
@jit(parallel=True)
def ONERDMFT_exchange_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M//2):
        for b in range(0,M//2):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2,M):
        for b in range(M//2,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return -1/2.*energy

In [7]:
@jit(parallel=True)
def ONERDMFT_Mueller_exchange_correlation_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M//2):
        for b in range(0,M//2):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2,M):
        for b in range(M//2,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return -1/2.*energy

In [10]:
print("El. Ec_{GU} Ec_{Mu} Ec_{FCI} Etot_{GU} Etot_{Mu} Etot_{FCI}")
for el in ['Li']:
    mol = gto.Mole()
    mol.atom = f"""
        {el}    0.    0.    0.
    """
    # this basis has 2 functions for Helium
    #mol.basis = "6-31g" 
    mol.basis = "ccpvtz"
    #mol.basis = "sto-6g"
    mol.spin = 1 
    mol.verbose=0
    mol.build()
    
    # the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
    eri = mol.intor('int2e')
    S = mol.intor('int1e_ovlp')
        
    ## Run Hartree-Fock.
    mf = scf.RHF(mol)
    mf.kernel()
    
    C = mf.mo_coeff
    h = mf.get_hcore()
    
    N = mol.nelec[0]
    P=np.matmul(C[:,0:N],C[:,0:N].T)
    
    gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)
    occu, naturalC = np.linalg.eigh(gamma)
    # need to first make gamma and then expand into blockdiagonal so "fake" alpha and "beta" are seperated in this fashion
    CE = expand_matrix(C)
    gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)
    occu_aa, naturalC_aa = np.linalg.eigh(gamma)
    occu = np.append(occu_aa, occu_aa)
    naturalCE = expand_matrix(naturalC_aa)
    naturalCTT = np.matmul(CE,naturalCE)
    for i, n  in enumerate(occu):
        if n < 0:
            occu[i] = 0 
    fs = fci.FCI(mol, mf.mo_coeff)
    e, ci = fs.kernel(verbose=0)
    FCIgamma_a, FCIgamma_b = fci.direct_spin1.make_rdm1s(ci, mf.mo_coeff.shape[0], mol.nelec)
    FCIoccu_a, FCInaturalC_a = np.linalg.eigh(FCIgamma_a)
    FCIoccu_b, FCInaturalC_b = np.linalg.eigh(FCIgamma_b)
    FCInaturalCTT_a, FCInaturalCTT_b = np.matmul(C,FCInaturalC_a), np.matmul(C,FCInaturalC_b)
    print(FCIoccu_a, FCIoccu_b)
    # FCInaturalCTTE = expand_matrix(FCInaturalCTT)
    # FCIoccuE = np.append(FCIoccu,FCIoccu)
    # #print(FCInaturalCTTE)
    # E_H = ONERDMFT_hartree_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
    # E_U = ONERDMFT_Umrigar_hartree_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
    # E_x = ONERDMFT_exchange_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
    # GU_E_xc = ONERDMFT_Umrigar_exchange_correlation_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
    # GU_E_c = (2*np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T)))) + E_U + GU_E_xc) - mf.e_tot
    # GU_tot = (2*np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T)))) + E_U + GU_E_xc)
    # Mu_E_xc = ONERDMFT_Mueller_exchange_correlation_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
    # Mu_E_c = (2*np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T)))) + E_H + Mu_E_xc) - mf.e_tot
    # Mu_tot = (2*np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T)))) + E_H + Mu_E_xc)
    # FCI_c = e-mf.e_tot
    # FCI_tot = e
    # print(f"{el:2s} {GU_E_c:1.6f} {Mu_E_c:1.6f} {FCI_c:1.6f} {GU_tot:3.6f} {Mu_tot:3.6f} {FCI_tot:3.6f}")

El. Ec_{GU} Ec_{Mu} Ec_{FCI} Etot_{GU} Etot_{Mu} Etot_{FCI}
[1.11463381e-08 1.11463730e-08 1.11463737e-08 1.41711545e-08
 1.41711644e-08 1.41711792e-08 1.41711935e-08 1.41712094e-08
 1.41712574e-08 1.41712632e-08 6.00902308e-08 6.00902474e-08
 6.00902750e-08 6.00902807e-08 6.00902922e-08 6.62446885e-07
 6.62446956e-07 6.62447006e-07 6.62447127e-07 6.62447223e-07
 2.00142524e-06 2.10747497e-05 2.10747499e-05 2.10747501e-05
 9.99642540e-05 9.99642567e-05 9.99642655e-05 1.53453420e-03
 9.98371337e-01 9.99725265e-01] [3.10580208e-11 3.10581155e-11 3.10581745e-11 8.33329381e-10
 8.33329723e-10 8.33331157e-10 8.33332197e-10 8.33332471e-10
 1.73354541e-09 1.41848281e-08 1.41848380e-08 1.41848529e-08
 1.41848671e-08 1.41848838e-08 1.41849316e-08 1.41849378e-08
 2.82828543e-07 2.82828800e-07 2.82828986e-07 6.05660205e-07
 6.05660296e-07 6.05660317e-07 6.05660466e-07 6.05660561e-07
 3.67660335e-06 9.17890493e-05 9.17890514e-05 9.17890570e-05
 1.52997248e-03 9.98187002e-01]
